In [1]:
# Import packages
import os
import sys
import csv
import re
import random
import shutil
from collections import defaultdict
from collections import Counter
import numpy as np
from tqdm import tqdm
import pickle

In [2]:
# Paths
sc13t2_dir = '../data/sc13t2/'
train_data_fpath = os.path.join(sc13t2_dir, 'train.tsv')
test_data_fpath = os.path.join(sc13t2_dir, 'test.tsv')

In [3]:
# Helper functions
# regex_sym = re.compile('[-_\.\\+/&#> \'\(\)\[\]\:\%\!\,\*]')
regex_sym = re.compile('[-_\.\\+/&#> \']')
def normalize_abbr_variation(abbr):
    abbr = abbr.strip()
    abbr = regex_sym.sub('', abbr)
    abbr = abbr.lower()
    return abbr

In [4]:
# Load training/test abbrs
train_group_cui_dict, test_group_cui_dict = defaultdict(set), defaultdict(set)
for data_fpath, group_cui_dict, data_name in [(train_data_fpath, train_group_cui_dict, 'train'),
                                             (test_data_fpath, test_group_cui_dict, 'test')]:
    with open(data_fpath, 'r', encoding='utf-8') as fd:
        reader = csv.reader(fd, delimiter='\t', quotechar=None)
        for i, l in enumerate(reader):
            if i == 0: continue
            abbr, cui = l[1], l[4]
            group_cui_dict[normalize_abbr_variation(abbr)].add(cui)
        print(f'Read {i} {data_name} records')
print(f'Train {len(train_group_cui_dict)} abbr groups ({sum([len(v) for v in train_group_cui_dict.values()])} CUIs) read')
print(f'Test {len(test_group_cui_dict)} abbr groups ({sum([len(v) for v in test_group_cui_dict.values()])} CUIs) read')

test_examples_dict = {}
with open(test_data_fpath, 'r', encoding='utf-8') as fd:
    reader = csv.reader(fd, delimiter='\t', quotechar=None)
    for i, l in enumerate(reader):
        if i == 0: continue
        guid, abbr, cui_label, cuis_cand = l[0], l[1], l[4], [c for c in l[5:] if c.strip()]
        test_examples_dict[int(guid)] = [abbr, cui_label, cuis_cand]

Read 3805 train records
Read 3774 test records
Train 670 abbr groups (860 CUIs) read
Test 704 abbr groups (884 CUIs) read


In [5]:
# Evaluation Functions
def evaluate_test(result_fpath):
    with open(result_fpath, 'rb') as fd:
        results = pickle.load(fd)
    results_dict = {r['guid']:r for r in results}
    
    total_cnt, total_correct, seen_cnt, seen_correct, lrabr_cnt, lrabr_correct, nolbl_cnt = (0,) * 7
    
    for guid, (abbr, label_cui, cand_cuis) in test_examples_dict.items():
        total_cnt += 1
        if guid in results_dict:
            r = results_dict[guid]
            cand_log_probs = [r[f'log_probs_train_{i}'].sum() / len(r[f'log_probs_train_{i}']) \
                              for i in range(len(cand_cuis))]
            output_cui = cand_cuis[np.argmax(cand_log_probs)]
            correct = int(output_cui == label_cui)
            total_correct += correct
            if normalize_abbr_variation(abbr) in train_group_cui_dict:
                seen_cnt += 1
                seen_correct += correct
            else:
                lrabr_cnt += 1
                lrabr_correct += correct
        else:
            nolbl_cnt += 1
            
    # print((total_cnt, total_correct, seen_cnt, seen_correct, lrabr_cnt, lrabr_correct, nolbl_cnt))
    print(f'Total : {total_correct/total_cnt} ({total_correct}/{total_cnt})')
    print(f'Seen  : {seen_correct/seen_cnt} ({seen_correct}/{seen_cnt})')
    print(f'Unseen: {lrabr_correct/(lrabr_cnt+nolbl_cnt)} ({lrabr_correct}/{(lrabr_cnt+nolbl_cnt)})')
    print(f'   - LRABR augmented: {lrabr_correct/lrabr_cnt} ({lrabr_correct}/{lrabr_cnt})')
    print(f'   - No labels: {0.0} (0/{nolbl_cnt})')
    print(f'Seen + LRABR: {(seen_correct+lrabr_correct)/(seen_cnt+lrabr_cnt)} ({seen_correct+lrabr_correct}/{seen_cnt+lrabr_cnt})')

In [6]:
evaluate_test('../results/exp_sc13t2_masklm2_out/output.pkl')

Total : 0.766295707472178 (2892/3774)
Seen  : 0.8446511627906976 (2724/3225)
Unseen: 0.30601092896174864 (168/549)
   - LRABR augmented: 0.4386422976501306 (168/383)
   - No labels: 0.0 (0/166)
Seen + LRABR: 0.8015521064301552 (2892/3608)


## Some Stat & Random / Majority Accuracy

In [7]:
# LRABR example stats
result_fpath = '../results/exp_sc13t2_masklm2_out/output.pkl'
with open(result_fpath, 'rb') as fd:
    results = pickle.load(fd)
results_dict = {r['guid']:r for r in results}

lrabr_cnt, lrabr_cands, lrabr_possible, lrabr_cands_possible = 0, 0, 0, 0
for guid, (abbr, label_cui, cand_cuis) in test_examples_dict.items():
    if guid in results_dict:
        if normalize_abbr_variation(abbr) not in train_group_cui_dict:
            lrabr_cnt += 1
            lrabr_cands += len(cand_cuis)
            if label_cui in cand_cuis:
                lrabr_possible += 1
                lrabr_cands_possible += len(cand_cuis)
print(f'LRABR examples: {lrabr_cnt}')
print(f'All cand CUIs in LRABR examples: {lrabr_cands}')
print('')
print(f'LRABR examples that have GT: {lrabr_possible}')
print(f'All cand CUIs in LRABR examples that have GT: {lrabr_cands_possible}')

LRABR examples: 383
All cand CUIs in LRABR examples: 2317

LRABR examples that have GT: 187
All cand CUIs in LRABR examples that have GT: 990


In [8]:
train_abbr_cuis = defaultdict(list)

with open(train_data_fpath, 'r') as fd:
    reader = csv.reader(fd, delimiter='\t', quotechar=None)
    for i, line in enumerate(reader):
        if i == 0:
#             print(line)
            abbr_idx = line.index('group')
            label_idx = line.index('label')
            continue
        abbr = line[abbr_idx]
        label = line[label_idx]
        train_abbr_cuis[normalize_abbr_variation(abbr)].append(label)
        
train_majority_cuis = {}
for abbr, cuis in train_abbr_cuis.items():
    counter = Counter(cuis)
    train_majority_cuis[abbr] = counter.most_common()[0][0]

In [9]:
test_total_cnt = 0
test_majority_cnt = 0
test_random_cnt = 0

with open(test_data_fpath, 'r') as fd:
    reader = csv.reader(fd, delimiter='\t', quotechar=None)
    for i, line in enumerate(reader):
        if i == 0:
#             print(line)
            abbr_idx = line.index('group')
            label_idx = line.index('label')
            continue
        abbr = line[abbr_idx]
        label = line[label_idx]
        cands = []
        for j in range(label_idx+1, len(line)):
            if line[j]:
                cands.append(line[j])
        
        test_total_cnt += 1
        if label in cands:
            test_random_cnt += 1.0/len(cands)
        if normalize_abbr_variation(abbr) in train_majority_cuis and \
                    train_majority_cuis[normalize_abbr_variation(abbr)] == label:
            test_majority_cnt += 1

In [10]:
print(test_total_cnt)
print(test_majority_cnt)
print(test_random_cnt)

print(f'Random  : {test_random_cnt/test_total_cnt}')
print(f'Majority: {test_majority_cnt/test_total_cnt}')

3774
2614
2100.0549575423483
Random  : 0.5564533538797956
Majority: 0.6926338102808691
